In [ ]:
# mount drive into colab
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# change directory
# For Marzieh
#%cd '/content/drive/MyDrive/Rahnema-College-Team'

# For Sanaz
%cd '/content/drive/MyDrive/Rahnema/Rahnema-College-Team'

/content/drive/.shortcut-targets-by-id/1mN07jhPqKcfkME6GxvaJOdBZWSwOA7AN/Rahnema-College-Team


In [ ]:
# unzip the log file
#!gzip -d output.log.gz

***
# Imports

In [ ]:
%%capture
!pip install pyyaml ua-parser user-agents
!pip install maxminddb-geolite2

In [ ]:
import pandas as pd
pd.options.display.max_columns = None
import re
from datetime import datetime
from user_agents import parse
from sklearn.preprocessing import OrdinalEncoder
from geolite2 import geolite2

***
# Read Log File

In [ ]:
# name of the columns
columns = ['ip', 'datetime', 'method', 'request', 'status', 'size', 'agent_time']

In [ ]:
# read log file and load it into a dataframe
def logs_to_df(logfile):
  
  with open(logfile) as source_file:
    lines = source_file.readlines()
    parsed_lines = []
    
    for i,line in enumerate(lines):
      parsed_lines.append(line.split(' ', 6))
  
  return pd.DataFrame(parsed_lines, columns=columns)

In [ ]:
df = logs_to_df(logfile='output.log')
df.head()

,ip,datetime,method,request,status,size,agent_time
0,207.213.193.143,[2021-5-12T5:6:0.0+0430],[Get,/cdn/profiles/1026106239],304,0,[[Googlebot-Image/1.0]] 32\n
1,207.213.193.143,[2021-5-12T5:6:0.0+0430],[Get,images/badge.png],304,0,[[Googlebot-Image/1.0]] 4\n
2,35.110.222.153,[2021-5-12T5:6:0.0+0430],[Get,/pages/630180847],200,52567,[[Mozilla/5.0 (Linux; Android 6.0.1; SAMSUNG S...
3,35.108.208.99,[2021-5-12T5:6:0.0+0430],[Get,images/fav_icon2.ico],200,23531,[[Mozilla/5.0 (Linux; Android 6.0; CAM-L21) Ap...
4,35.110.222.153,[2021-5-12T5:6:0.0+0430],[Get,images/sanjagh_logo_purpule5.png],200,4680,[[Mozilla/5.0 (Linux; Android 6.0.1; SAMSUNG S...


In [ ]:
print('Size of dataframe:', df.shape)

Size of dataframe: (1260035, 7)


In [ ]:
# split user_agent and response_time columns from eachother
df[['user_agent','response_time']] = pd.DataFrame(df['agent_time'].str.rsplit(' ',1).tolist(),
                                                  columns=['user_agent','response_time'])

# delete agent_time column
del df['agent_time']

In [ ]:
# remove unwanted characters from each column
df['datetime'] = df['datetime'].str.replace('\[', '')
df['datetime'] = df['datetime'].str.replace('\]', '')
df['method'] = df['method'].str.replace('\[', '')
df['request'] = df['request'].str.replace('\]', '')
df['user_agent'] = df['user_agent'].str.replace('\[\[', '')
df['user_agent'] = df['user_agent'].str.replace('\]\]', '')
df['response_time'] = df['response_time'].str.replace('\n', '')

In [ ]:
# number of missing values in each column
df.isnull().sum()

ip               0
datetime         0
method           0
request          0
status           0
size             0
user_agent       0
response_time    0
dtype: int64

### Change data type

In [ ]:
# change datetime column data type to datetime
df['datetime'] = pd.to_datetime(df['datetime'])
# change method column data type to category
df['method'] = df['method'].astype('category')
# change status column data type to int
df['status'] = df['status'].astype('int16')
# change size column data type to int
df['size'] = df['size'].astype('int32')
# change user_agent column data type to category
df['user_agent'] = df['user_agent'].astype('category')

### Extract time and date periods

In [ ]:
# extract date periods from datetime field
df['date'] = df['datetime'].dt.date
df['year'] = df['datetime'].dt.year
df['quarter'] = df['datetime'].dt.quarter
df['month'] = df['datetime'].dt.month
df['day'] = df['datetime'].dt.day

# extract time periods from datetime field
df['time'] = df['datetime'].dt.time
df['hour'] = df['datetime'].dt.hour
df['minute'] = df['datetime'].dt.minute
df['second'] = df['datetime'].dt.second

# remove datetime column
#del df['datetime']

In [ ]:
df.head()

,ip,datetime,method,request,status,size,user_agent,response_time,date,year,quarter,month,day,time,hour,minute,second
0,207.213.193.143,2021-05-12 05:06:00+04:30,Get,/cdn/profiles/1026106239,304,0,Googlebot-Image/1.0,32,2021-05-12,2021,2,5,12,05:06:00,5,6,0
1,207.213.193.143,2021-05-12 05:06:00+04:30,Get,images/badge.png,304,0,Googlebot-Image/1.0,4,2021-05-12,2021,2,5,12,05:06:00,5,6,0
2,35.110.222.153,2021-05-12 05:06:00+04:30,Get,/pages/630180847,200,52567,Mozilla/5.0 (Linux; Android 6.0.1; SAMSUNG SM-...,32,2021-05-12,2021,2,5,12,05:06:00,5,6,0
3,35.108.208.99,2021-05-12 05:06:00+04:30,Get,images/fav_icon2.ico,200,23531,Mozilla/5.0 (Linux; Android 6.0; CAM-L21) Appl...,20,2021-05-12,2021,2,5,12,05:06:00,5,6,0
4,35.110.222.153,2021-05-12 05:06:00+04:30,Get,images/sanjagh_logo_purpule5.png,200,4680,Mozilla/5.0 (Linux; Android 6.0.1; SAMSUNG SM-...,8,2021-05-12,2021,2,5,12,05:06:00,5,6,0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1260035 entries, 0 to 1260034
Data columns (total 17 columns):
 #   Column         Non-Null Count    Dtype                                
---  ------         --------------    -----                                
 0   ip             1260035 non-null  object                               
 1   datetime       1260035 non-null  datetime64[ns, pytz.FixedOffset(270)]
 2   method         1260035 non-null  category                             
 3   request        1260035 non-null  object                               
 4   status         1260035 non-null  int16                                
 5   size           1260035 non-null  int32                                
 6   user_agent     1260035 non-null  category                             
 7   response_time  1260035 non-null  object                               
 8   date           1260035 non-null  object                               
 9   year           1260035 non-null  int64        

***
# Data Cleaning

In [ ]:
# seperate subnet from ip addresss
for i in range(len(df)):
  df.at[i,'subnet'] = df['ip'][i].rsplit('.', 1)[0]

In [ ]:
# extract first and last part and depth of the request
for i in range(len(df)):
  split_request = df['request'][i].split('/')
  split_request = [item for item in split_request if item!='']

  if len(split_request):
    df.at[i,'first_part_request'] = split_request[0]
    df.at[i,'last_part_request'] = split_request[-1]
    df.at[i,'depth_request'] = len(split_request)
  
  else:
    df.at[i,'first_part_request'] = 'root'
    df.at[i,'last_part_request'] = 'root'
    df.at[i,'depth_request'] = 0

In [ ]:
# remove ips with - as value
df = df[df['ip'] != '-']

# replace response time of - with 0
df.loc[df['response_time'] == '-', 'response_time'] = '0'

print('Size of new dataframe:', df.shape)

# change ip column data type to category
df['ip'] = df['ip'].astype('category')

# change response_time column data type to int
df['response_time'] = df['response_time'].astype('int64')

Size of new dataframe: (1241945, 21)


### status 

In [ ]:
# extract the first character of status to group them as 2xx, 3xx, 4xx and 5xx statuses
df['status_group'] = df['status'].astype(str).str[0] + 'xx' 

In [ ]:
df[['status' ,'status_group']].head()

,status,status_group
0,304,3xx
1,304,3xx
2,200,2xx
3,200,2xx
4,200,2xx


### country 

In [ ]:
# get country names from ip
def get_country(ip):
    try:
        x = geo.get(ip)
    except ValueError:
        return pd.np.nan
    try:
        return x['country']['names']['en'] if x else pd.np.nan
    except KeyError:
        return pd.np.nan

geo = geolite2.reader()
# get unique IPs
unique_ips = df['ip'].unique()
# make series out of it
unique_ips = pd.Series(unique_ips, index = unique_ips)
# map IP --> country
df['country'] = df['ip'].map(unique_ips.apply(get_country))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
df[['ip', 'country']].head()

,ip,country
0,207.213.193.143,United States
1,207.213.193.143,United States
2,35.110.222.153,United States
3,35.108.208.99,United States
4,35.110.222.153,United States


## parse user_agent

In [ ]:
# user_agent parsing
browser_families, browser_versions, os_families = [], [], []
os_versions, device_families, device_brands, is_bots = [], [], [], []

for i,item in enumerate(df['user_agent'].to_list()):
  ua_string = item

  try:
    user_agent = parse(ua_string)

    # Accessing user agent's browser attributes
    browser_families.append(user_agent.browser.family)
    browser_versions.append(user_agent.browser.version_string)

    # Accessing user agent's operating system properties
    os_families.append(user_agent.os.family)
    os_versions.append(user_agent.os.version_string)

    # Accessing user agent's device properties
    device_families.append(user_agent.device.family)
    device_brands.append(user_agent.device.brand)

    # whether user agent is a search engine crawler/spider
    is_bots.append(user_agent.is_bot)
  
  except:
    browser_families.append(' ')
    browser_versions.append(' ')

    # Accessing user agent's operating system properties
    os_families.append(' ')
    os_versions.append(' ')

    # Accessing user agent's device properties
    device_families.append(' ')
    device_brands.append(' ')
    
    # whether user agent is a search engine crawler/spider
    is_bots.append(' ')

In [ ]:
df['browser_family'] = browser_families
df['browser_version'] = browser_versions
df['os_family'] = os_families
df['os_version'] = os_versions 
df['device_family'] = device_families
df['device_brand'] = device_brands
df['is_bot'] = [int(val) for val in is_bots]

In [ ]:
# concatenate ip and user_agent columns as single id
df['id'] = df['ip'].str.cat(df['user_agent'], sep =' ')
df['id'].head()

0                  207.213.193.143 Googlebot-Image/1.0
1                  207.213.193.143 Googlebot-Image/1.0
2    35.110.222.153 Mozilla/5.0 (Linux; Android 6.0...
3    35.108.208.99 Mozilla/5.0 (Linux; Android 6.0;...
4    35.110.222.153 Mozilla/5.0 (Linux; Android 6.0...
Name: id, dtype: object

## robots.txt

In [ ]:
# search "robots.txt" in request
df['robots'] = df.request.str.match('.*robots.txt', case=False)

# change True = 1 , False = 0 
ord_enc = OrdinalEncoder()
df["robots"] = ord_enc.fit_transform(df[["robots"]])

# show a request "robots.txt"
df['robots'].head()

0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: robots, dtype: float64

In [ ]:
df.head()

,ip,datetime,method,request,status,size,user_agent,response_time,date,year,quarter,month,day,time,hour,minute,second,subnet,first_part_request,last_part_request,depth_request,status_group,country,browser_family,browser_version,os_family,os_version,device_family,device_brand,is_bot,id,robots
0,207.213.193.143,2021-05-12 05:06:00+04:30,Get,/cdn/profiles/1026106239,304,0,Googlebot-Image/1.0,32,2021-05-12,2021,2,5,12,05:06:00,5,6,0,207.213.193,cdn,1026106239,3.0,3xx,United States,Googlebot-Image,1.0,Other,,Spider,Spider,1,207.213.193.143 Googlebot-Image/1.0,0.0
1,207.213.193.143,2021-05-12 05:06:00+04:30,Get,images/badge.png,304,0,Googlebot-Image/1.0,4,2021-05-12,2021,2,5,12,05:06:00,5,6,0,207.213.193,images,badge.png,2.0,3xx,United States,Googlebot-Image,1.0,Other,,Spider,Spider,1,207.213.193.143 Googlebot-Image/1.0,0.0
2,35.110.222.153,2021-05-12 05:06:00+04:30,Get,/pages/630180847,200,52567,Mozilla/5.0 (Linux; Android 6.0.1; SAMSUNG SM-...,32,2021-05-12,2021,2,5,12,05:06:00,5,6,0,35.110.222,pages,630180847,2.0,2xx,United States,Samsung Internet,4.0,Android,6.0.1,Samsung SM-J710GN,Samsung,0,35.110.222.153 Mozilla/5.0 (Linux; Android 6.0...,0.0
3,35.108.208.99,2021-05-12 05:06:00+04:30,Get,images/fav_icon2.ico,200,23531,Mozilla/5.0 (Linux; Android 6.0; CAM-L21) Appl...,20,2021-05-12,2021,2,5,12,05:06:00,5,6,0,35.108.208,images,fav_icon2.ico,2.0,2xx,United States,Chrome Mobile,91.0.4472,Android,6.0,Huawei CAM-L21,Huawei,0,35.108.208.99 Mozilla/5.0 (Linux; Android 6.0;...,0.0
4,35.110.222.153,2021-05-12 05:06:00+04:30,Get,images/sanjagh_logo_purpule5.png,200,4680,Mozilla/5.0 (Linux; Android 6.0.1; SAMSUNG SM-...,8,2021-05-12,2021,2,5,12,05:06:00,5,6,0,35.110.222,images,sanjagh_logo_purpule5.png,2.0,2xx,United States,Samsung Internet,4.0,Android,6.0.1,Samsung SM-J710GN,Samsung,0,35.110.222.153 Mozilla/5.0 (Linux; Android 6.0...,0.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1241945 entries, 0 to 1260034
Data columns (total 32 columns):
 #   Column              Non-Null Count    Dtype                                
---  ------              --------------    -----                                
 0   ip                  1241945 non-null  category                             
 1   datetime            1241945 non-null  datetime64[ns, pytz.FixedOffset(270)]
 2   method              1241945 non-null  category                             
 3   request             1241945 non-null  object                               
 4   status              1241945 non-null  int16                                
 5   size                1241945 non-null  int32                                
 6   user_agent          1241945 non-null  category                             
 7   response_time       1241945 non-null  int64                                
 8   date                1241945 non-null  object                            

***
# Save log file as CSV file

In [ ]:
# save dataframe to a CSV file
df.to_csv('logs_cleaned.csv', index=False)